In [2]:
import os
os.getcwd()
os.chdir('/Users/chidam/Desktop')

In [3]:
#Viterbi algorithm - HMM - going with the original algorithm without altering
import math
from nltk.corpus import stopwords
import pandas as pd
import re
from collections import defaultdict
import nltk
#from nltk.corpus import stopwords

def reading_file(document): 
    filtered_sentence=[]
    res = defaultdict(list)
    df=pd.read_csv(document, names=['City', 'Details'], header=None, encoding='latin-1')
    #df=pd.read_csv('/Users/chidam/Desktop/data_trial_hmm_viterbi.csv', names=['City', 'Details'], header=None)
    df['Details'] = df['Details'].str.replace(',', ' ')
    #print(df.head())
    for i, j in df.itertuples(index=False):
        #print(i)
        s=re.split(r'(?<=\.) ', j )
        tag_sent=[nltk.word_tokenize(i) for i in s]
        pos_tag_sent=[nltk.pos_tag(sent) for sent in tag_sent]
        #res[i].append([[i] for i in s])
        #print(tag_sent)
        res[i].append(pos_tag_sent)
    return (res)


def hmm_viterbi(doc):
    len_tagged_doc=len(doc)
    #print(len_tagged_doc)
#     for line in doc:
#         len_tagged_doc=[x for x in line]
    transition_prob={}
    word_tag_prob={}
    max_of_tag_for_word={}
    
    tag_list=[]
    #tag_list*=len_tagged_doc
    word_list=[]
    #word_list*=len_tagged_doc
    result=[]

    for k in doc:#this is the key in the dictionary like New York
        for vita in doc[k]:#this is the blob of text - here it's in the form of list os lists containing tuples of words and tags
            for line in vita:
                result.append(line)
                for elem in line:
                    word_list.append(elem[0])
                    tag_list.append(elem[1])
                    
    for i in range (len(tag_list)):
        if (i+1) < (len(tag_list)):
            current_tag=tag_list[i+1]#inner: inner key of the transition_prob dictionary 
            #print(current_tag)
            
            previous_tag=tag_list[i]#outer: outer key of the transition_prob dictionary
            transition_prob[previous_tag]=transition_prob.get(previous_tag, {})
            transition_prob[previous_tag][current_tag]=transition_prob[previous_tag].get(current_tag, 0)
            transition_prob[previous_tag][current_tag]+=1
        
        #setting up the word-tag dictionary to get the word likelihood probability
            word_key=word_list[i]#outer: outer key of the word_tag_prob dictionary
            tag_key=tag_list[i]#inner: inner key of the word_tag_prob dictionary
            word_tag_prob[word_key]=word_tag_prob.get(word_key, {})
            word_tag_prob[word_key][tag_key]=word_tag_prob[word_key].get(tag_key, 0)
            word_tag_prob[word_key][tag_key]+=1
        
#         #to account for first word after '.'
    transition_prob['.'] = transition_prob.get('.',{})
    transition_prob['.'][tag_list[0]] = transition_prob['.'].get(tag_list[0],0)
        #print(tag_list[0])
    transition_prob['.'][tag_list[0]]+=1
    #print(transition_prob['.'][tag_list[0]])
        
        #last word_tag pair
    last_word_tag_index=len_tagged_doc-1
        
    last_word_key=word_list[last_word_tag_index]#outer key of the word_tag_prob dictionary
    last_tag_key=tag_list[last_word_tag_index]#inner key of the word_tag_prob dictionary
    word_tag_prob[last_word_key]=word_tag_prob.get(last_word_key, {})
    word_tag_prob[last_word_key][last_tag_key]=word_tag_prob[last_word_key].get(last_tag_key, 0)
    word_tag_prob[last_word_key][last_tag_key]+=1

    #calculating probabilities using the count-values from the dictionaries and assigning them back to the dictionary.so now we have probabilities for tags dictionary instead of counts  
    for k in transition_prob:
        ans=transition_prob[k]
        s=sum(ans.values())
    #print("all the values of the key {} are: {}". format(k, s))
        for j in ans:
            ans[j]= round(ans[j]/s, 1)
        ans=ans.items()
        ans=sorted(ans,key=lambda x: x[0])
        transition_prob[k]=ans

    
    #calculating probabilities for word-tag dictionary (it's just a dictionary with words and their associated tags. here i find probability of the tags associated with their words. if a word has more than one tag means the word has appeared more than once!)
    for key in word_tag_prob:
        aw=word_tag_prob[key]
        max_of_tag_for_word[key]=max(aw, key=aw.get)
        su=sum(aw.values())
        for ele in aw:
            aw[ele]= round(aw[ele]/su, 1)
        aw=aw.items()
        aw=sorted(aw, key=lambda x: x[0])
        word_tag_prob[key]=aw
    
    
    
#     by transition_prob i mean transition from one state(tag) to another state and probabilities of the same    
#     max_of_tag_for_word : (meaning getting the tag that has maximum probability and assigning that to the word from every word-tag list)
    return( word_tag_prob, transition_prob, word_list, tag_list, result)
    #return( word_tag_prob)
        
#reading_file('/Users/chidam/Desktop/data_trial_hmm_viterbi.csv')
hmm_viterbi(reading_file('/Users/chidam/Desktop/Barcelona.csv'))

({'local': [('JJ', 1.0)],
  'put': [('VBD', 1.0)],
  'really': [('RB', 1.0)],
  'Magos': [('NNP', 1.0)],
  'inhabitants': [('NNS', 1.0)],
  'falls': [('VBZ', 1.0)],
  'legs': [('NNS', 1.0)],
  'covered': [('VBN', 1.0)],
  'craziest': [('JJS', 1.0)],
  'away': [('RB', 0.5), ('RP', 0.5)],
  'European': [('JJ', 1.0)],
  'create': [('VB', 1.0)],
  'which': [('WDT', 1.0)],
  'popular': [('JJ', 1.0)],
  'student': [('NN', 1.0)],
  'times': [('NNS', 1.0)],
  'spring/early': [('JJ', 1.0)],
  'fruits': [('NNS', 1.0)],
  'let': [('VB', 1.0)],
  'arm': [('NN', 1.0)],
  'engine': [('NN', 1.0)],
  'Sitges': [('NNP', 1.0)],
  'wondrous': [('JJ', 1.0)],
  'Circuit-Festival-Barcelona': [('NN', 1.0)],
  'resort': [('NN', 1.0)],
  'early': [('JJ', 0.6), ('RB', 0.4)],
  'raised': [('VBN', 1.0)],
  'making': [('VBG', 1.0)],
  'nasty': [('JJ', 1.0)],
  'decorations': [('NNS', 1.0)],
  'strain': [('NN', 1.0)],
  '12am': [('CD', 1.0)],
  'ways': [('NNS', 1.0)],
  'beaten': [('NN', 1.0)],
  "n't": [('RB', 1.0

In [4]:
from collections import defaultdict
from operator import itemgetter

def viterbi(values_from_hmm):#output from the above function: hmm_viterbi are assigned to the following four variables
    my_dict = defaultdict(list)
    emission_probability=values_from_hmm[0]
    transition_probability=values_from_hmm[1]
    observations=values_from_hmm[2]#words
    states=values_from_hmm[3]#tags
    word_tag_from_doc=values_from_hmm[4]
    
    new_path = []
    already_path=[]
    prob_previous=0
    previous_tag=None
    result_newpath=[]
    
 
    #print(states)
    
    # Add the probabilities of beginning the sequence with each possible state  
        
#     for i in range(len(word_tag_from_doc)):#i need res to get the list of items - i have to loop through that list of lists
        #print(i)
    for e in word_tag_from_doc:#i use this to have access to the list of lists
        already_path.append(e)
        for (a, b) in e:
                #print(a)
                #print(a)
            if a == e[0][0]:#meaning if a is the first word of every list
                if a in emission_probability:#i use this because this is the one that's calculated by using HMM
                    for itc in emission_probability[a]:
                            #print(itc)
                        present_tag=itc[0]#corresponding current tag according to hmm word_tag directory
                            #print(present_tag)
                        present_val=itc[1]#corresponding current tag value of the word that i am looping through    
                        max_cal=[]
                        max_cal.append(((present_val*1), present_tag))
                        #already_path.append((a,present_tag))
#                             print(already_path)
                    anw = sorted(max_cal, key=lambda x: x, reverse=True)[0]#check whether the value was output in a list if yes add [0]                    
                        #print(anw)
                    previous_tag=anw[1]
                            #print(previous_tag)
                    prob_previous=anw[0]
                            #print(prob_previous)
                        #already_path.append(((a,previous_tag)))
                    new_path.append([])
                    i = len(new_path)-1
                    new_path[i].append((a, previous_tag))
                        #print(already_path)
                      
            else:         
                if a in emission_probability:#i use this because this is the one that's calculated by using HMM
                    for itc in emission_probability[a]:
                            #print(itc)
                        present_tag=itc[0]#corresponding current tag according to hmm word_tag directory
                            #print(present_tag)
                        present_val=itc[1]#corresponding current tag value of the word that i am looping through
                            #print(present_val)
                            #print(transition_probability[previous_tag])
                            #already_path.append((a,present_tag))
                        try:
                            ans=float([item[1] for item in transition_probability[previous_tag] if present_tag in item][0])
                            #print(ans)
                        except IndexError: ## not the best way to code...do not catch exceptions, but this is an exception to the exception catching rule
                            ans=0.1
                        max_cal=[]
                        max_cal.append(((present_val*prob_previous*ans), present_tag))#sample
                    
                    anw = sorted(max_cal, key=lambda x: x, reverse=True)[0]#check whether the value was output in a list if yes add [0]                    
                            #print(anw)
                    previous_tag=anw[1]
                    prob_previous=anw[0]
                        #print(prob_previous)
                        #print(previous_tag) 
                        #already_path.append((a, previous_tag))
                        
                    new_path[i].append((a, previous_tag))
            
    return(new_path)

In [7]:
#extracting information/relationship-using nltk.ne_chunk

from nltk.sem.relextract import tree2semi_rel, semi_rel2reldict
import pandas as pd
from datetime import date
document=viterbi(hmm_viterbi(reading_file('/Users/chidam/Desktop/Barcelona.csv')))
chunk_result=[]
for i in document:
    res=nltk.ne_chunk(i)
    chunk_result.append(semi_rel2reldict(tree2semi_rel(res)))
    #print(chunk_result)
result=[]
for veg in chunk_result:
    for carrot in veg:
        result.append([])
        i = len(result)-1
        result[i].append((carrot['untagged_filler']))
        #carrot['subjtext'], carrot['objtext'], carrot['rcon'], 
result

#this section is write the results into a file:
# with open ('/Users/chidam/Desktop/Barcelona.txt', 'a') as f:
#     for line in result:   
#         f.write(str(line))

[["usually around 4:30pm after that these extravagantly dressed gentlemen set off on the calvacada del Reis a grand parade of floats replete with acrobats clowns and elves from which the kings dish out sweets to local kids the route normally starts at the lower entrance of parc-de-la-ciutadella running up carrer-de-marques del'Argentera-Via Laietana ."],
 ['February/March'],
 ["'s carnival is back and growing in popularity year by year as in most"],
 ['countries carnival in'],
 ["is the last chance to go wild before Lent when forty days of abstinence kick in the culmination of the party is la-Gran-Rua-de-Carneval when visitors can expect to see plenty of fancy dress floats and fireworks form a grand parade down one of the city 's main streets normally av-paral-lel the real action takes place down the road in"],
 ['countryside slaying dragons but occasionally he would be called upon in these warmer climes to dispatch a nasty lizard or two Such heroic behaviour saw him dubbed patron sain